In [1]:
import folium
import pandas as pd

In [2]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [7]:
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
spacex_df = pd.read_csv(URL)

In [8]:
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [9]:
## Task 1: Mark all launch sites on a map

In [10]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [11]:
#We first need to create a folium Map object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [12]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [13]:
# Task 2: Mark the success/failed launches for each site on the map
# Define colors for success and failure
success_color = 'green'
failure_color = 'red'

for idx, row in spacex_df.iterrows():
    # Extract the latitude, longitude, class (success=1/failure=0), and launch site
    lat = row['Lat']
    lon = row['Long']
    launch_site = row['Launch Site']
    launch_class = row['class']
    
    # Choose color based on success/failure
    color = success_color if launch_class == 1 else failure_color
    
    # Create a marker with a popup that shows the launch site and status
    folium.CircleMarker(
        location=[lat, lon],
        radius=7,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=f"{launch_site} - {'Success' if launch_class == 1 else 'Failure'}"
    ).add_to(site_map)

# Finally display the map
site_map

In [14]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [16]:
# Initialize the MarkerCluster object
marker_cluster = MarkerCluster()

# Function to get marker color based on class
def get_marker_color(launch_class):
    if launch_class == 1:
        return 'green'
    else:
        return 'red'

# Iterate through each row and add a marker with appropriate color to the cluster
for idx, row in spacex_df.iterrows():
    lat = row['Lat']
    lon = row['Long']
    launch_site = row['Launch Site']
    launch_class = row['class']
    marker_color = get_marker_color(launch_class)
    
    # Create marker with popup
    marker = folium.Marker(
        location=[lat, lon],
        popup=f"{launch_site} - {'Success' if launch_class == 1 else 'Failure'}",
        icon=folium.Icon(color=marker_color)
    )
    
    # Add marker to the cluster
    marker_cluster.add_child(marker)

# Add the MarkerCluster to the map
site_map.add_child(marker_cluster)

# Display the map
site_map

In [17]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)

site_map

In [18]:
## TASK 3: Calculate the distances between a launch site to its proximities

import numpy as np

def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees).
    Returns distance in kilometers.
    """
    # Convert decimal degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    r = 6371  # Radius of Earth in kilometers
    return c * r


In [19]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [20]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [22]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
# Example list of coastline points (lat, lon)
coastline_coords = [
    (28.56367, -80.57163),  # Example point 1
    (29.0, -81.0),          # Example point 2
    (30.0, -79.5),          # Example point 3
    # Add as many as needed...
]


def find_closest_coastline(launch_lat, launch_lon, coastline_coords):
    min_distance = float('inf')
    closest_coord = None
    
    for coast_lat, coast_lon in coastline_coords:
        dist = calculate_distance(launch_lat, launch_lon, coast_lat, coast_lon)
        if dist < min_distance:
            min_distance = dist
            closest_coord = (coast_lat, coast_lon)
            
    return closest_coord, min_distance

# Example usage for a launch site:
launch_site_lat = 28.573255  # Replace with actual launch site latitude
launch_site_lon = -80.646895  # Replace with actual launch site longitude

closest_coast, distance_to_coast = find_closest_coastline(launch_site_lat, launch_site_lon, coastline_coords)

print(f"Closest coastline coordinate: {closest_coast}")
print(f"Distance to coastline: {distance_to_coast:.2f} km")


Closest coastline coordinate: (28.56367, -80.57163)
Distance to coastline: 7.43 km


In [23]:
from folium.features import DivIcon

# Coordinates
launch_site_coordinate = (launch_site_lat, launch_site_lon)  # e.g., (28.573255, -80.646895)
coastline_coordinate = closest_coast  # e.g., (28.56367, -80.57163)

# Add marker for launch site (optional, if not already on the map)
folium.Marker(
    location=launch_site_coordinate,
    popup="Launch Site",
    icon=folium.Icon(color='blue', icon='rocket', prefix='fa')
).add_to(site_map)

# Add marker for closest coastline point
distance_marker = folium.Marker(
    location=coastline_coordinate,
    icon=DivIcon(
        icon_size=(100, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{}</b></div>'.format(f"{distance_to_coast:.2f} KM")
    )
)
site_map.add_child(distance_marker)

# Optionally, draw a line between launch site and coastline
folium.PolyLine(
    locations=[launch_site_coordinate, coastline_coordinate],
    color='orange',
    weight=2,
    dash_array='5, 5'
).add_to(site_map)

# Display the map
site_map


In [24]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site
from folium.features import DivIcon

# Example launch site coordinate
launch_site_coord = (launch_site_lat, launch_site_lon)

# Example closest POIs — these should come from your data or manual input
closest_pois = {
    'City': (28.5355, -80.6217),
    'Railway': (28.5500, -80.6500),
    'Highway': (28.5600, -80.6000),
}

# Example distances to POIs in km — you can calculate these using your calculate_distance function
distances_to_pois = {
    'City': 5.23,
    'Railway': 3.85,
    'Highway': 7.40,
}

# Add launch site marker (optional)
folium.Marker(
    location=launch_site_coord,
    popup="Launch Site",
    icon=folium.Icon(color='blue', icon='rocket', prefix='fa')
).add_to(site_map)

for poi_type, poi_coord in closest_pois.items():
    distance = distances_to_pois[poi_type]
    
    # Add marker for POI with distance label
    poi_marker = folium.Marker(
        location=poi_coord,
        icon=DivIcon(
            icon_size=(100, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12px; color:#1f618d;"><b>{poi_type}: {distance:.2f} KM</b></div>'
        ),
        popup=f"{poi_type} - {distance:.2f} KM"
    )
    site_map.add_child(poi_marker)
    
    # Draw a line between launch site and POI
    folium.PolyLine(
        locations=[launch_site_coord, poi_coord],
        color='purple',
        weight=2,
        dash_array='5, 5'
    ).add_to(site_map)

site_map  # Show the map
